### loading dataset

In [16]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
data = pd.read_csv(r'../2. data_cleaning\4.1 some cols FEed.csv')
data.head(3)

,property_name,sector,society,area_in_sqft,price_in_crores,price_per_sqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating,property_type,noOfFloor,area
0,2 BHK Flat in Sector 70 Gurgaon,sector 70,shree vardhman victoria,1300.0,1.10,8461.0,Carpet area: 1300 (120.77 sq.m.),2.0,2.0,2,not available,"Sector 70 Gurgaon, Gurgaon, Haryana",8.0,north-east,1 to 5 Year Old,"['Airia Mall', 'Golf Course Extn Road', 'Sohna...",Square yards rera reg.\r\nHrera 660/2017/307\r...,"['3 Wardrobe', '1 Modular Kitchen', '1 Chimney...","['Security / Fire Alarm', 'Power Back-up', 'Fe...","['Green Area5 out of 5', 'Construction5 out of...",flat,1.0,NaN
1,3 Bedroom House for sale in Sector 40 Gurgaon,sector 40,NaN,900.0,2.15,23889.0,Plot area 100(83.61 sq.m.),3.0,3.0,3,"Pooja Room,Study Room,Store Room","Sector 40 Gurgaon, Gurgaon, Haryana",0.0,NaN,5 to 10 year old,"['Huda city centre metro station', 'Axis bank ...",Its just behind ryan international school and ...,"['4 Wardrobe', '5 Fan', '4 Geyser', '2 Light',...","['Feng Shui / Vaastu Compliant', 'Private Gard...","['Environment4 out of 5', 'Lifestyle4 out of 5...",house,3.0,(84 sq.m.) Plot Area
2,5 Bedroom House for sale in Palam Vihar,palam vihar,ansals palam vihar,4680.0,9.00,19231.0,Plot area 520(434.79 sq.m.),5.0,5.0,2,"Pooja Room,Study Room,Servant Room,Store Room","J-Block, Palam Vihar, Gurgaon, Haryana",0.0,north,10+ year old,"['Palam Vihar Vyapar kendra', 'Palam triangle'...",North facing plot aviable on sale in palam vih...,"['1 Water Purifier', '4 Fan', '1 Fridge', '1 E...","['Feng Shui / Vaastu Compliant', 'Private Gard...","['Environment5 out of 5', 'Safety4 out of 5', ...",house,2.0,(435 sq.m.) Plot Area


### FEing `areaWithType` col

In [35]:
data['areaWithType'].head(3)

0    Carpet area: 1300 (120.77 sq.m.)
1          Plot area 100(83.61 sq.m.)
2         Plot area 520(434.79 sq.m.)
Name: areaWithType, dtype: object

#### `area_in_sqft` col contains one of the 3 area_types in `areaWithType` col.

In [18]:
data[['area_in_sqft', 'areaWithType']].sample(3)

,area_in_sqft,areaWithType
2615,1600.0,Carpet area: 1600 (148.64 sq.m.)
2161,1875.0,Super Built up area 1875(174.19 sq.m.)
1131,1530.0,Super Built up area 1350(125.42 sq.m.)


#### creating `superBuiltupArea` col

In [19]:
data['areaWithType'].str.lower().str.strip().sample(3)

1660    super built up area 2191(203.55 sq.m.)carpet a...
3435               super built up area 1573(146.14 sq.m.)
3395                super built up area 1050(97.55 sq.m.)
Name: areaWithType, dtype: object

In [20]:
data['SuperBuiltupArea'] = data['areaWithType'].str.lower().str.strip().str.extract(r'super built up area (\d+\.?\d+)')
data['SuperBuiltupArea']

0        NaN
1        NaN
2        NaN
3       2010
4        NaN
        ... 
4077     NaN
4078    1780
4079     NaN
4080     NaN
4081     NaN
Name: SuperBuiltupArea, Length: 4082, dtype: object

#### creating `BuiltupArea` col

In [21]:
data['BuiltupArea'] = data['areaWithType'].str.lower().str.strip().str.extract(r'built up area\s*:\s*(\d+\.?\d*)')
data['BuiltupArea']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
4077    NaN
4078    NaN
4079    NaN
4080    NaN
4081    NaN
Name: BuiltupArea, Length: 4082, dtype: object

#### creating `CarpetArea` col

In [22]:
data['CarpetArea'] = data['areaWithType'].str.lower().str.strip().str.extract(r'carpet area\s*:\s*(\d+\.?\d*)')
data['CarpetArea']

0       1300
1        NaN
2        NaN
3        NaN
4       3500
        ... 
4077     NaN
4078     NaN
4079    2295
4080    1120
4081    1300
Name: CarpetArea, Length: 4082, dtype: object

#### converting all 3 area_cols from sqm to sqft

In [23]:
import re
def convert_to_sqft(area_text, area_value):
    if not area_value:
        return None
    match = re.search(r"{} \((\d+\.?\d*) sq.m.\)*".format(area_value), str(area_text))
    if match:
        sqm_value = float(match.group(1))
        return sqm_value*10.76391
    return area_value

In [24]:
data['CarpetArea'] = data.apply(lambda x: convert_to_sqft(x['areaWithType'], x['CarpetArea']), axis=1)
data['BuiltupArea'] = data.apply(lambda x: convert_to_sqft(x['areaWithType'], x['BuiltupArea']), axis=1)
data['SuperBuiltupArea'] = data.apply(lambda x: convert_to_sqft(x['areaWithType'], x['SuperBuiltupArea']), axis=1)

In [33]:
data[['SuperBuiltupArea', 'BuiltupArea',	'CarpetArea']].head(3)

,SuperBuiltupArea,BuiltupArea,CarpetArea
0,NaN,NaN,1299.957411
1,NaN,NaN,NaN
2,NaN,NaN,NaN


#### checking where none of the 3 are null

In [26]:
data[~(data['CarpetArea'].isnull() | data['BuiltupArea'].isnull() | data['SuperBuiltupArea'].isnull())][['CarpetArea', 'BuiltupArea', 'SuperBuiltupArea']].sample(5)

,CarpetArea,BuiltupArea,SuperBuiltupArea
2894,2250,2500,3090
662,1650,1900,2149
3619,2398,2399,2408
805,500,556,567
1201,1752,2343,2344


In [27]:
data[(data['CarpetArea'].isnull() & data['BuiltupArea'].isnull() & data['SuperBuiltupArea'].isnull())][['CarpetArea', 'BuiltupArea', 'SuperBuiltupArea']].shape

(735, 3)

In [28]:
data[(data['CarpetArea'].isnull() & data['BuiltupArea'].isnull() & data['SuperBuiltupArea'].isnull())][['CarpetArea', 'BuiltupArea', 'SuperBuiltupArea']].head()

,CarpetArea,BuiltupArea,SuperBuiltupArea
1,NaN,NaN,NaN
2,NaN,NaN,NaN
9,NaN,NaN,NaN
10,NaN,NaN,NaN
12,NaN,NaN,NaN
